In [1]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
from pathlib import Path
import geopandas as gpd
from scipy import spatial

In [2]:
# Import BlueBikes Stations
filepath_bluebikes_stations = ".././02 Data/BlueBikes/"
bluebikes_stations_df = pd.read_csv(filepath_bluebikes_stations + 'current_bluebikes_stations.csv', header=1)
bluebikes_stations_df.head()

,Number,Name,Latitude,Longitude,District,Public,Total docks,Deployment Year
0,K32015,1200 Beacon St,42.344149,-71.114674,Brookline,Yes,15,2021.0
1,W32006,160 Arsenal,42.364664,-71.175694,Watertown,Yes,11,2021.0
2,A32019,175 N Harvard St,42.363796,-71.129164,Boston,Yes,18,2014.0
3,S32035,191 Beacon St,42.380323,-71.108786,Somerville,Yes,19,2018.0
4,C32094,2 Hummingbird Lane at Olmsted Green,42.288870,-71.095003,Boston,Yes,17,2020.0


In [4]:
bluebikes_stations_df['District'].unique()

array(['Brookline', 'Watertown', 'Boston', 'Somerville', 'Cambridge',
       'Everett', 'Chelsea', 'Newton', 'Arlington', 'Salem', nan,
       'Revere'], dtype=object)

In [5]:
district_list = ['Cambridge','Somerville']
bluebikes_stations_cs_df=bluebikes_stations_df[bluebikes_stations_df['District'].isin(district_list)].reset_index().drop('index',axis=1)
bluebikes_stations_cs_df

,Number,Name,Latitude,Longitude,District,Public,Total docks,Deployment Year
0,S32035,191 Beacon St,42.380323,-71.108786,Somerville,Yes,19,2018.0
1,S32023,30 Dane St,42.381001,-71.104025,Somerville,Yes,15,2018.0
2,M32026,359 Broadway - Broadway at Fayette Street,42.370803,-71.104412,Cambridge,Yes,23,2013.0
3,M32054,699 Mt Auburn St,42.375002,-71.148716,Cambridge,Yes,24,2018.0
4,M32060,700 Huron Ave,42.380788,-71.154129,Cambridge,Yes,19,2018.0
...,...,...,...,...,...,...,...,...
103,M32040,University Park,42.362648,-71.100061,Cambridge,Yes,18,2015.0
104,M32057,Vassal Lane at Tobin/VLUS,42.385368,-71.137757,Cambridge,Yes,19,2018.0
105,M32050,Verizon Innovation Hub 10 Ware Street,42.372509,-71.113054,Cambridge,Yes,19,2017.0
106,S32021,Washington St at Myrtle St,42.381751,-71.083952,Somerville,Yes,15,2018.0


In [6]:
# Filter the trips that start and end in the desired area
lat_min = 42.35217552009351 
lat_max = 42.38008690523708 
lon_min = -71.12714994786728    
lon_max = -71.08267810377765 

bluebikes_stations_tt_df = bluebikes_stations_cs_df[
    bluebikes_stations_cs_df["Latitude"].between(lat_min, lat_max) & 
    bluebikes_stations_cs_df["Longitude"].between(lon_min, lon_max)]

bluebikes_stations_tt_df = bluebikes_stations_tt_df.reset_index().drop('index',axis=1)

In [7]:
bluebikes_stations_tt_df.Latitude = bluebikes_stations_tt_df.Latitude.round(6)
bluebikes_stations_tt_df

,Number,Name,Latitude,Longitude,District,Public,Total docks,Deployment Year
0,M32026,359 Broadway - Broadway at Fayette Street,42.370803,-71.104412,Cambridge,Yes,23,2013.0
1,M32073,955 Mass Ave,42.368952,-71.109988,Cambridge,Yes,19,2021.0
2,M32059,Ames St at Broadway,42.363693,-71.087567,Cambridge,Yes,19,2018.0
3,M32037,Ames St at Main St,42.362500,-71.088220,Cambridge,Yes,19,2012.0
4,S32003,Beacon St at Washington / Kirkland,42.378754,-71.107072,Somerville,Yes,19,2012.0
5,M32067,Berkshire Street at Cambridge Street,42.372077,-71.089954,Cambridge,Yes,19,2019.0
6,M32027,Binney St / Sixth St,42.366088,-71.086336,Cambridge,Yes,19,2013.0
7,M32052,Cambridge Dept. of Public Works -147 Hampshire...,42.371197,-71.097599,Cambridge,Yes,19,2017.0
8,M32013,Cambridge Main Library at Broadway / Trowbridg...,42.373379,-71.111075,Cambridge,Yes,19,2012.0
9,M32007,Cambridge St - at Columbia St / Webster Ave,42.372969,-71.094445,Cambridge,Yes,15,2012.0


In [8]:
# Save the results of bluebikes stations in Cambridge
bluebikes_path = Path('./Input Database/bluebikes_stations_tangibletable.csv')
bluebikes_stations_tt_df.to_csv(bluebikes_path,index=False)